## Исследование структуры LSTM моделей

### Подключение библиотек

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

import sys
import os
sys.path.append(os.path.abspath('..'))
from helpful_functions import train_val_test_split, \
                              create_sequences,\
                              train_and_evaluate_model

### Подготовка данных

In [5]:
file_path = '../dataset/preprocessed_data.csv'
df = pd.read_csv(file_path)

df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df.sort_index(inplace=True)

target_feature = 'Global_active_power'
test_ratio = 0.10
val_ratio = 0.10

train_data, val_data, test_data = train_val_test_split(df, val_ratio, test_ratio)

scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_data)
val_scaled = scaler.transform(val_data)
test_scaled = scaler.transform(test_data)

target_idx = df.columns.get_loc(target_feature)

features_count = train_data.shape[1]

### Обучение моделей с разными структурами

In [6]:
lr = 0.01
window_size = 30

X_train, y_train = create_sequences(train_scaled, target_idx, window_size)
X_val, y_val = create_sequences(val_scaled, target_idx, window_size)
X_test, y_test = create_sequences(test_scaled, target_idx, window_size)

lstm_architectures = [
    [50],
    [100],
    [150],
    [50, 50],
    [50, 100],
    [100, 50],
    [100, 100],
    [100, 150],
    [150, 150]
]

results = {}

for arch in lstm_architectures:
    model_name = "LSTM_" + "_".join(map(str, arch))
    print(f"\nОбучение модели: {model_name}")

    lstm_model = Sequential()
    lstm_model.add(Input(shape=(window_size, features_count)))

    for i, units in enumerate(arch):
        return_sequences = i < len(arch) - 1 
        lstm_model.add(LSTM(units, activation='tanh', return_sequences=return_sequences))

    lstm_model.add(Dense(1))

    optimizer = Adam(learning_rate=lr)
    lstm_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    result = train_and_evaluate_model(
        lstm_model, model_name, X_train, y_train, X_val, y_val, X_test, y_test,
        lr, window_size, scaler, target_feature, df.columns
    )

    results[model_name] = result


Обучение модели: LSTM_50
LSTM_50 | val_loss: 0.0004 | MSE_val: 0.0444

Обучение модели: LSTM_100
LSTM_100 | val_loss: 0.0004 | MSE_val: 0.0429

Обучение модели: LSTM_150
LSTM_150 | val_loss: 0.0003 | MSE_val: 0.0424

Обучение модели: LSTM_50_50
LSTM_50_50 | val_loss: 0.0003 | MSE_val: 0.0409

Обучение модели: LSTM_50_100
LSTM_50_100 | val_loss: 0.0003 | MSE_val: 0.0396

Обучение модели: LSTM_100_50
LSTM_100_50 | val_loss: 0.0003 | MSE_val: 0.0389

Обучение модели: LSTM_100_100
LSTM_100_100 | val_loss: 0.0004 | MSE_val: 0.0432

Обучение модели: LSTM_100_150
LSTM_100_150 | val_loss: 0.0003 | MSE_val: 0.0414

Обучение модели: LSTM_150_150
LSTM_150_150 | val_loss: 0.0003 | MSE_val: 0.0403


### Вывод ошибок всех структур моделей

In [12]:
results_df = pd.DataFrame(results.values())
print(results_df.sort_values(by=['mse_val_dn']))

     model_type  learning_rate  window_size  train_loss  val_loss  mse_val_dn  \
5   LSTM_100_50           0.01           30    0.000418  0.000319    0.038893   
4   LSTM_50_100           0.01           30    0.000430  0.000325    0.039609   
8  LSTM_150_150           0.01           30    0.000422  0.000330    0.040311   
3    LSTM_50_50           0.01           30    0.000426  0.000335    0.040850   
7  LSTM_100_150           0.01           30    0.000425  0.000339    0.041356   
2      LSTM_150           0.01           30    0.000453  0.000347    0.042391   
1      LSTM_100           0.01           30    0.000453  0.000352    0.042949   
6  LSTM_100_100           0.01           30    0.000431  0.000354    0.043171   
0       LSTM_50           0.01           30    0.000457  0.000364    0.044413   

   mse_test_dn  
5     0.033188  
4     0.033370  
8     0.032949  
3     0.035816  
7     0.035750  
2     0.034993  
1     0.036102  
6     0.037436  
0     0.037440  


### Вывод информации о лучшей структуре модели

In [15]:
lstm_results = results_df[results_df['model_type'].str.startswith('LSTM_')]
best_lstm = lstm_results.nsmallest(1, 'mse_val_dn').iloc[0]

print("\nЛучшая LSTM (по валидации):")
print(best_lstm[['model_type', 'learning_rate', 'window_size', 'mse_val_dn', 'mse_test_dn']])


Лучшая LSTM (по валидации):
model_type       LSTM_100_50
learning_rate           0.01
window_size               30
mse_val_dn          0.038893
mse_test_dn         0.033188
Name: 5, dtype: object
